# Example Code for Estimating the Ground State Energy of Hydroxyl (·OH)

## Basic Installation

Install required package, we highly recommend participant to use qiskit platform, or at least participants can finish preprocessing at other platform and transfer the circuit to qiskit format, since our noise model is from IBM real machine backend and we restricted some algorithmic seeds which could be varied from different platform.

In [1]:
!pip install qiskit
!pip install qiskit-nature[pyscf] -U

In [2]:
!pip install qiskit_aer

In [3]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper,ParityMapper,QubitConverter
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
import numpy as np
import pylab
import qiskit.providers
from qiskit import Aer,pulse, QuantumCircuit
from qiskit.utils import QuantumInstance, algorithm_globals
import time

Here we require paticipants to fix the algorithm seed in qiskit. *MUST* translate other format circuit to qiskit before any place need algorithm seed. And we give 20, 21, 30, 33, 36, 42, 43, 55, 67, 170 as seeds that requires to run, and the result will be calculated as the average of results from each seed. And please use shots as 4000.

In [4]:
seeds = 170
algorithm_globals.random_seed = seeds
seed_transpiler = seeds
iterations = 125
shot = 4000

## Generate Hamiltonian and Pauli String

At this step, the example code uses PySCF to generate the hamiltonian of hydroxyl with basis function as 'sto3g' to fit the spin orbital, then uses JordanWignerMapper to map the fermionic terms to pauli strings. To be noticed, other chemistry tool also allowed to be used at this step, but keep in mind to use 'sto-3g' and Jordan Wigner Mapper which should gives 12 qubits and 631 paulil terms.

In [5]:
ultra_simplified_ala_string = """
O 0.0 0.0 0.0
H 0.45 -0.1525 -0.8454
"""

driver = PySCFDriver(
    atom=ultra_simplified_ala_string.strip(),
    basis='sto3g',
    charge=1,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)
qmolecule = driver.run()

/common/home/zl606/.conda/envs/qiskit43/lib/python3.11/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [6]:
hamiltonian = qmolecule.hamiltonian
coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)
second_q_op = hamiltonian.second_q_op()

Polynomial Tensor
 "+-":
[[-3.21461222e+01  5.59899100e-01  1.87617178e-01  1.01255650e-15
   8.11480219e-16 -1.94702445e-01]
 [ 5.59899100e-01 -7.35898345e+00 -2.46352634e-01 -1.30936709e-15
  -2.55042478e-15  9.51226718e-01]
 [ 1.87617178e-01 -2.46352634e-01 -6.56995119e+00  3.21526376e-15
   4.26166795e-15 -1.09726793e+00]
 [ 1.01240566e-15 -1.30835286e-15  3.05517355e-15 -6.94886145e+00
   8.05414387e-16 -5.99383933e-15]
 [ 8.12281085e-16 -2.67843588e-15  3.91912265e-15  1.05277494e-15
  -6.94886145e+00 -7.55119945e-15]
 [-1.94702445e-01  9.51226718e-01 -1.09726793e+00 -6.05046761e-15
  -7.43333720e-15 -4.64967973e+00]]
 "++--":
[[[[ 4.74977044e+00 -4.38465691e-01 -1.51436760e-01 -8.26158100e-16
    -6.26098781e-16  1.59790984e-01]
   [-4.38465691e-01  6.47204045e-02  1.84429506e-02  1.06166485e-16
     9.65364155e-17 -2.66865302e-02]
   [-1.51436760e-01  1.84429506e-02  2.46189939e-02 -1.53212272e-17
    -2.23430157e-17  6.40512562e-03]
   [-8.25955988e-16  1.05875759e-16 -1.54695

In [7]:
mapper = JordanWignerMapper()
converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
qubit_op = converter.convert(second_q_op)

/tmp/ipykernel_1017741/561404713.py:2: DeprecationWarning: The QubitConverter class is deprecated as of version 0.6.0 and will be removed no sooner than 3 months after the release . Instead you should directly use the QubitMapper instance which you used to pass into the QubitConverter as the first argument. Refer to the documentation of the qiskit_nature.second_q.mappers module for more information.
  converter = QubitConverter(mapper=mapper, two_qubit_reduction=False)
/common/home/zl606/.conda/envs/qiskit43/lib/python3.11/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


We recommend to use classical minimum eigensolver to obtain a reference energy at this step. In case some of the classical minimum eigensolver donot directly gives nuclear repulsion energy, we give reference energies below: *Comupted Energy*: -78.75252123, *Nuclear Repulsion_energy*: 4.36537496654537. *Obtained Reference Ground State Energy*: -74.38714627.

In [8]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [9]:
result = solver.solve(qmolecule)
print(result.computed_energies)

[-78.75252123]


In [10]:
print(result.nuclear_repulsion_energy)

4.36537496654537


In [11]:
ref_value = result.computed_energies + result.nuclear_repulsion_energy
print(ref_value)

[-74.38714627]


### Load the Noise Models.

Load three noise models: ibmq_kolkata, ibmq_montreal, ibmq_cairo

In [12]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from qiskit import *
import time
from qiskit.providers.aer.noise import NoiseModel
import qiskit.providers.aer.noise as noise
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.providers.fake_provider import *
import pickle
from qiskit_aer.primitives import Sampler
from qiskit import qasm2


In [13]:
with open('NoiseModel/fakekolkata.pkl', 'rb') as file:
    noise_model_kolkata = pickle.load(file)
with open('NoiseModel/fakecairo.pkl', 'rb') as file:
    noise_model_cairo = pickle.load(file)
with open('NoiseModel/fakemontreal.pkl', 'rb') as file:
    noise_model_montreal = pickle.load(file)
noise_model1 = noise.NoiseModel()
noise_model2= noise.NoiseModel()
noise_model3 = noise.NoiseModel()
noise_model_kolkata = noise_model1.from_dict(noise_model_kolkata)
noise_model_cairo = noise_model2.from_dict(noise_model_cairo)
noise_model_montreal = noise_model3.from_dict(noise_model_montreal)

### Group measurement

This is the part for **pauli grouping**.

All the implementation is in `utils/varsaw.py`.

We use pauli_grouping to reduce the total 631 observables to only **142 observables**.

For each observable we take 4000 shots, we our total number of shots is 142*4000 = **568000**. It is under 1800000.

In [14]:
from utils.varsaw import parseHamiltonian, group_measurements, varsaw_expectation

In [15]:
import pickle
# run once (commented code)!
h, first_term = parseHamiltonian('Hamiltonian/OHhamiltonian.txt')
# measurements, measurement_dict = group_measurements(h)
# filehandler = open(b"142observables.obj","wb")
# pickle.dump((measurements, measurement_dict),filehandler)

filehandler = open(b"142observables.obj","rb")
measurements, measurement_dict = pickle.load(filehandler)


### Load all the Ansatzs.

Load 3 ansatzs from qasm file. The 3 ansatzs will be used for 3 different noise models.

In [16]:
circuit_montreal = qasm2.load('montreal.qasm')
# computed_energies_montreal = varsaw_expectation(circuit_montreal, measurements, measurement_dict, first_term, h, sampler_montreal)

circuit_kolkata = qasm2.load('kolkata.qasm')
# computed_energies_kolkata = varsaw_expectation(circuit_montreal, measurements, measurement_dict, first_term, h, sampler_kolkata)

circuit_cairo = qasm2.load('cairo.qasm')
# computed_energies_cairo = varsaw_expectation(circuit_montreal, measurements, measurement_dict, first_term, h, sampler_cairo)

####  Build 3 samplers with 3 noise models.

The samplers here have the same function as qiskit estimator. They both use noise models and take the same arguments like `shots` and `seed`.

We try different seeds here. For each seed, we build 3 samplers for 3 noise models. Then we test 3 ansatzs on the correspondent 3 samplers respectively.

It may takes around 20 mins.

In [17]:
for seeds in [20, 21, 30, 33, 36, 42, 43, 55, 67, 170]:
    algorithm_globals.random_seed = seeds
    seed_transpiler = seeds
    iterations = 125
    shot = 4000

    sampler_kolkata = Sampler(
        backend_options = {
            'method': 'statevector',
            'device': 'CPU',
            'noise_model': noise_model_kolkata
        },
        run_options = {
            'shots': shot,
            'seed': seeds,
        },
        transpile_options = {
            'seed_transpiler':seed_transpiler
        }
    )
    sampler_montreal = Sampler(
        backend_options = {
            'method': 'statevector',
            'device': 'CPU',
            'noise_model': noise_model_montreal
        },
        run_options = {
            'shots': shot,
            'seed': seeds,
        },
        transpile_options = {
            'seed_transpiler':seed_transpiler
        }
    )
    sampler_cairo = Sampler(
        backend_options = {
            'method': 'statevector',
            'device': 'CPU',
            'noise_model': noise_model_cairo
        },
        run_options = {
            'shots': shot,
            'seed': seeds,
        },
        transpile_options = {
            'seed_transpiler':seed_transpiler
        }
    )
    computed_energies_montreal = varsaw_expectation(circuit_montreal, measurements, measurement_dict, first_term, h, sampler_montreal)

    computed_energies_kolkata = varsaw_expectation(circuit_montreal, measurements, measurement_dict, first_term, h, sampler_kolkata)

    computed_energies_cairo = varsaw_expectation(circuit_montreal, measurements, measurement_dict, first_term, h, sampler_cairo)
    
    # Calculate the Accuracy (Most Important Metric)
    
    print(f"======================Seed: {seeds}=========================")
    
    estimated = computed_energies_montreal + result.nuclear_repulsion_energy
    error_rate = abs(abs(ref_value - estimated) / ref_value * 100)
    print("Noise model: ibmq_montreal:")
    print("Error rate: %f%%" % (error_rate))
    print("Escore: %f" % (100-error_rate))

    estimated = computed_energies_kolkata + result.nuclear_repulsion_energy
    error_rate = abs(abs(ref_value - estimated) / ref_value * 100)
    print("Noise model: ibmq_kolkata:")
    print("Error rate: %f%%" % (error_rate))
    print("Escore: %f" % (100-error_rate))

    estimated = computed_energies_cairo + result.nuclear_repulsion_energy
    error_rate = abs(abs(ref_value - estimated) / ref_value * 100)
    print("Noise model: ibmq_cairo:")
    print("Error rate: %f%%" % (error_rate))
    print("Escore: %f" % (100-error_rate))

======================Seed: 20=========================
Noise model: ibmq_montreal:
Error rate: 2.468030%
Escore: 97.531970
Noise model: ibmq_kolkata:
Error rate: 1.967509%
Escore: 98.032491
Noise model: ibmq_cairo:
Error rate: 1.327241%
Escore: 98.672759
======================Seed: 21=========================
Noise model: ibmq_montreal:
Error rate: 2.468071%
Escore: 97.531929
Noise model: ibmq_kolkata:
Error rate: 1.967373%
Escore: 98.032627
Noise model: ibmq_cairo:
Error rate: 1.327053%
Escore: 98.672947
======================Seed: 30=========================
Noise model: ibmq_montreal:
Error rate: 2.466273%
Escore: 97.533727
Noise model: ibmq_kolkata:
Error rate: 1.973335%
Escore: 98.026665
Noise model: ibmq_cairo:
Error rate: 1.325874%
Escore: 98.674126
======================Seed: 33=========================
Noise model: ibmq_montreal:
Error rate: 2.467117%
Escore: 97.532883
Noise model: ibmq_kolkata:
Error rate: 1.973323%
Escore: 98.026677
Noise model: ibmq_cairo:
Error rate: 1.32

## Obtain the Duration of Quantum Circuit

In [18]:
from qiskit.providers.fake_provider import *
for name, system_model in [('montreal.qasm', FakeMontreal()), ('kolkata.qasm', FakeKolkata()), ('cairo.qasm', FakeCairo())]:
    transpiled_circuit = qasm2.load(name)
    with pulse.build(system_model) as my_program1:
        pulse.call(transpiled_circuit)
    print(f'duration on ibmq_{name} is {my_program1.duration}')


duration on ibmq_montreal.qasm is 160
duration on ibmq_kolkata.qasm is 160
duration on ibmq_cairo.qasm is 96


On above we have shown our experiment results. Next we will showcase our novel compilation framework for UCCSD ansatz.

You don't need to run the code below because it has nothing to do with the grading. We just put the code here for anybody who is interesed in our compilation framework.
### Appendix
##### Tetris: A Hardware-aware Compilation and Transpilation framework for UCCSD ansatz

Here we shows an example code of how to run Tetris to maximize the CNOT cancellation and reduce circuit duration. For more design details please refer to our self-reflection report.


In [19]:
from arch import load_coupling_map
from time import time as ctime
from utils.parallel_bl import gate_count_oriented_scheduling
from utils.synthesis_broccoli import synthesis
from qiskit import QuantumCircuit, transpile

def Tetris_Montreal(parr, use_bridge=False):
    print('Tetris passes, Our schedule, Our synthesis, montreal', flush=True)
    lnq = len(parr[0][0]) # lnq: number of qubits
    length = lnq // 2 # `length' is a hyperparameter, and can be adjusted for best performance. Here we keep `length' fixed for simplicity.
    coup = load_coupling_map('montreal')
    t0 = ctime()
    a2 = gate_count_oriented_scheduling(parr) # parr is the pauli_block list
    # a2 = [[block] for block in parr]
    qc, metrics = synthesis(a2, arch='montreal', use_bridge=use_bridge)
    pnq = qc.num_qubits
    print(pnq)
    print('Tetris, Time costed:', ctime()-t0, flush=True)
    t0 = ctime()
    qc2 = transpile(qc, basis_gates=['u3', 'cx'], coupling_map=coup, initial_layout=list(range(pnq)), optimization_level=3)
    print('Qiskit L3, Time costed:', ctime()-t0, flush=True)
    return qc2


In [20]:
import ast
from benchmark.mypauli import pauliString

pauli_blocks = []
for pauli_list in ansatz._operators: # load all pauli blocks to a list
    block = ast.literal_eval(pauli_list._primitive._pauli_list.__str__())
    block = [pauliString(ps) for ps in block]
    pauli_blocks.append(block)

new_ansatz = Tetris_Montreal(pauli_blocks, use_bridge=False)

NameError: name 'ansatz' is not defined

duration on ibmq_montreal.qasm is 160
duration on ibmq_kolkata.qasm is 160
duration on ibmq_cairo.qasm is 96
